# Boundary from Children Demo

This notebook tests and visualizes the `cell_boundary_from_children` function which computes a cell's boundary using its descendant cells at a finer resolution.

In [1]:
import h3
import folium
from h3_toolkit.utils import children_on_boundary_faces
from h3_toolkit.geom import cell_boundary_to_geojson, cell_boundary_from_children

print("Imports successful!")

Imports successful!


## 1. Basic Test

Pick a parent cell and compute its boundary from children at different resolutions.

In [11]:
# Create a parent cell at resolution 6
# Res 8 parent with C++ bindings is fast!
lat, lng, res = 37.7749, -122.4194, 8
parent = h3.latlng_to_cell(lat, lng, res)
print(f"Parent cell: {parent}")
print(f"Parent resolution: {h3.get_resolution(parent)}")

# Get boundary children at res 8
child_res =14
boundary_children = children_on_boundary_faces(parent, child_res)
print(f"\nBoundary children at res {child_res}: {len(boundary_children)}")

# Get merged boundary GeoJSON
boundary_geojson = cell_boundary_from_children(parent, child_res)
print(f"\nGeoJSON properties: {boundary_geojson['properties']}")

Parent cell: 8828308281fffff
Parent resolution: 8

Boundary children at res 14: 2184

GeoJSON properties: {'h3_index': '8828308281fffff', 'child_resolution': 14, 'num_boundary_cells': 2184}


## 2. Visualization: Compare Smooth vs Jagged Boundary

In [12]:
m = folium.Map(location=[lat, lng], zoom_start=13)

# Original smooth boundary (red dashed)
smooth_boundary = cell_boundary_to_geojson(parent)
folium.GeoJson(
    smooth_boundary,
    style_function=lambda x: {
        'color': 'red', 
        'fillOpacity': 0, 
        'weight': 4, 
        'dashArray': '10,5'
    },
    name='Smooth Boundary'
).add_to(m)

# Boundary from children (blue solid)
folium.GeoJson(
    boundary_geojson,
    style_function=lambda x: {
        'color': 'blue', 
        'fillOpacity': 0.15, 
        'weight': 2
    },
    name='Boundary from Children'
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

m

## 3. Show Individual Boundary Children

In [4]:
m2 = folium.Map(location=[37.775938728915946, -122.41795063018799], zoom_start=13)

# Parent boundary
folium.GeoJson(
    smooth_boundary,
    style_function=lambda x: {'color': 'red', 'fillOpacity': 0, 'weight': 3, 'dashArray': '5,5'}
).add_to(m2)

# Each boundary child
for child in boundary_children:
    child_geojson = cell_boundary_to_geojson(child)
    folium.GeoJson(
        child_geojson,
        style_function=lambda x: {'color': 'purple', 'fillOpacity': 0.3, 'weight': 1}
    ).add_to(m2)

m2

## 4. Different Resolutions Comparison

In [5]:
# Compare boundaries at different child resolutions
resolutions = [7, 8, 9]
colors = ['green', 'blue', 'orange']

m3 = folium.Map(location=[37.775938728915946, -122.41795063018799], zoom_start=13)

# Parent boundary
folium.GeoJson(
    smooth_boundary,
    style_function=lambda x: {'color': 'red', 'fillOpacity': 0, 'weight': 4, 'dashArray': '10,5'},
    name='Parent (smooth)'
).add_to(m3)

for res, color in zip(resolutions, colors):
    boundary = cell_boundary_from_children(parent, res)
    folium.GeoJson(
        boundary,
        style_function=lambda x, c=color: {'color': c, 'fillOpacity': 0.05, 'weight': 2},
        name=f'Res {res} ({boundary["properties"]["num_boundary_cells"]} cells)'
    ).add_to(m3)

folium.LayerControl().add_to(m3)
m3

## 5. Count Boundary Cells at Different Resolutions

In [6]:
print(f"Parent cell: {parent} (res {h3.get_resolution(parent)})")
print("\nBoundary children count by resolution:")
print("-" * 40)

for res in range(h3.get_resolution(parent) + 1, 11):
    children = children_on_boundary_faces(parent, res)
    print(f"Resolution {res}: {len(children):,} boundary cells")

Parent cell: 86283082fffffff (res 6)

Boundary children count by resolution:
----------------------------------------
Resolution 7: 6 boundary cells
Resolution 8: 24 boundary cells
Resolution 9: 78 boundary cells
Resolution 10: 240 boundary cells
